In [1]:
import os

from pydantic import BaseModel, Field
from IPython.display import Image, display
from typing import Annotated, TypedDict, List

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [2]:
from dotenv import load_dotenv

load_dotenv("/home/ubuntu/LLM-tutorials/keys.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
# State 정의
class State(TypedDict):
    # list 타입에 add_messages 적용(list 에 message 추가)
    messages: Annotated[list, add_messages]

In [4]:
search_tool = TavilySearchResults(max_results=5)
tools = [search_tool]
tool_node = ToolNode(tools)

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def chatbot(state:State):
    answer = llm.invoke(state["messages"])
    return {"messages" : [answer]} ## State를 정의할 때 Anntated로 add_messages 적용했으므로 자동으로 메세지 리스트에 추가됨.

In [7]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "tools")
graph_builder.add_edge("tools", END)

graph = graph_builder.compile()
# display(Image(graph.get_graph().draw_mermaid_png()))

In [8]:
messages = graph.invoke({"messages" : [("user", "대한민국 서울의 날씨는 어떤가요?")]})
print(messages)

{'messages': [HumanMessage(content='대한민국 서울의 날씨는 어떤가요?', additional_kwargs={}, response_metadata={}, id='40249fec-d4d1-4189-ab88-5760121a5cc2'), AIMessage(content='현재 서울의 날씨에 대한 실시간 정보는 제공할 수 없지만, 일반적으로 서울의 날씨는 계절에 따라 크게 변합니다. \n\n- **봄 (3월~5월)**: 기온이 점차 따뜻해지며, 꽃이 피는 시기입니다. 평균 기온은 10도에서 20도 사이입니다.\n- **여름 (6월~8월)**: 덥고 습한 날씨가 지속되며, 평균 기온은 25도에서 30도 이상 올라갑니다. 장마철이 있어 비가 자주 내립니다.\n- **가을 (9월~11월)**: 기온이 서서히 내려가고, 단풍이 아름답게 물드는 시기입니다. 평균 기온은 10도에서 20도 사이입니다.\n- **겨울 (12월~2월)**: 춥고 건조한 날씨가 이어지며, 평균 기온은 0도 이하로 떨어질 수 있습니다. 눈이 내리는 경우도 있습니다.\n\n정확한 날씨 정보는 기상청이나 날씨 앱을 통해 확인하시는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 250, 'prompt_tokens': 18, 'total_tokens': 268, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44a

In [9]:
for message in messages["messages"]:
    print(message)

content='대한민국 서울의 날씨는 어떤가요?' additional_kwargs={} response_metadata={} id='40249fec-d4d1-4189-ab88-5760121a5cc2'
content='현재 서울의 날씨에 대한 실시간 정보는 제공할 수 없지만, 일반적으로 서울의 날씨는 계절에 따라 크게 변합니다. \n\n- **봄 (3월~5월)**: 기온이 점차 따뜻해지며, 꽃이 피는 시기입니다. 평균 기온은 10도에서 20도 사이입니다.\n- **여름 (6월~8월)**: 덥고 습한 날씨가 지속되며, 평균 기온은 25도에서 30도 이상 올라갑니다. 장마철이 있어 비가 자주 내립니다.\n- **가을 (9월~11월)**: 기온이 서서히 내려가고, 단풍이 아름답게 물드는 시기입니다. 평균 기온은 10도에서 20도 사이입니다.\n- **겨울 (12월~2월)**: 춥고 건조한 날씨가 이어지며, 평균 기온은 0도 이하로 떨어질 수 있습니다. 눈이 내리는 경우도 있습니다.\n\n정확한 날씨 정보는 기상청이나 날씨 앱을 통해 확인하시는 것이 좋습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 250, 'prompt_tokens': 18, 'total_tokens': 268, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BMUOTGGHYIMqakDKTs

In [10]:
def router_node(state:State):
    if messages:=state["messages"]:
        ai_message = messages[-1]
    else:
        raise ValueError("No messages found in state")
    
    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tools"
    
    # 도구 호출이 없는 경우 "END" 반환
    return END    

In [11]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges(
    source="chatbot",
    path=router_node,
    path_map={
        "tools" : "tools",
        END : END
    }
)

graph = graph_builder.compile()
# display(Image(graph.get_graph().draw_mermaid_png()))

In [12]:
messages = graph.invoke({"messages" : [("user", "구글이 뭐죠?")]})
print(messages)

{'messages': [HumanMessage(content='구글이 뭐죠?', additional_kwargs={}, response_metadata={}, id='b6a7e50e-86d0-4e6b-b132-c12884c325af'), AIMessage(content='구글(Google)은 미국의 다국적 기술 기업으로, 주로 인터넷 관련 서비스와 제품을 제공합니다. 1998년에 래리 페이지와 세르게이 브린에 의해 설립되었으며, 검색 엔진으로 시작하여 현재는 다양한 분야로 사업을 확장하였습니다. 구글의 주요 서비스에는 웹 검색, 광고 서비스(구글 애드워즈), 클라우드 컴퓨팅, 소프트웨어(안드로이드 운영체제, 구글 크롬 브라우저 등), 그리고 하드웨어(구글 픽셀 스마트폰, 구글 홈 등)가 포함됩니다.\n\n구글은 또한 유튜브(동영상 공유 플랫폼), 구글 맵스(지도 서비스), 구글 드라이브(클라우드 저장소) 등 다양한 인기 서비스를 운영하고 있습니다. 구글은 정보 접근성을 높이고, 사용자에게 유용한 도구와 서비스를 제공하는 것을 목표로 하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 13, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_80cf447eee', 'id': 'chatcmpl-BMUOuvTIBKo2ASt6S6

In [13]:
for message in messages["messages"]:
    print(message)

content='구글이 뭐죠?' additional_kwargs={} response_metadata={} id='b6a7e50e-86d0-4e6b-b132-c12884c325af'
content='구글(Google)은 미국의 다국적 기술 기업으로, 주로 인터넷 관련 서비스와 제품을 제공합니다. 1998년에 래리 페이지와 세르게이 브린에 의해 설립되었으며, 검색 엔진으로 시작하여 현재는 다양한 분야로 사업을 확장하였습니다. 구글의 주요 서비스에는 웹 검색, 광고 서비스(구글 애드워즈), 클라우드 컴퓨팅, 소프트웨어(안드로이드 운영체제, 구글 크롬 브라우저 등), 그리고 하드웨어(구글 픽셀 스마트폰, 구글 홈 등)가 포함됩니다.\n\n구글은 또한 유튜브(동영상 공유 플랫폼), 구글 맵스(지도 서비스), 구글 드라이브(클라우드 저장소) 등 다양한 인기 서비스를 운영하고 있습니다. 구글은 정보 접근성을 높이고, 사용자에게 유용한 도구와 서비스를 제공하는 것을 목표로 하고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 202, 'prompt_tokens': 13, 'total_tokens': 215, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_80cf447eee', 'id': 'chatcmpl-BMUOuvTIBKo2ASt6S60jPpWtRugxq', 'finish_reason': 'stop', 'logp